In [ ]:
%pip install numpy pandas nltk datasketch

## Load and Preprocess the Datasets

In [ ]:
import pandas as pd

# Load the train and test datasets
train_data = pd.read_csv('train.csv', sep=',')
test_data = pd.read_csv('test_without_labels.csv', sep=',')

# Display dataset sizes
print(f'Train Set Size: {len(train_data)}')
print(f'Test Set Size: {len(test_data)}')

Train Set Size: 111795
Test Set Size: 47912


In [ ]:
import nltk
from nltk.corpus import stopwords

# Load stopwords from nltk and convert to set for fast lookup
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [3]:
import re

# Preprocessing function - Remove punctuation, numbers and stopwords
def preprocess_text(text):
    if pd.isna(text):   # Handle NaN cases
        return ''
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)  # Remove punctuation
    text = re.sub(r'\d+', ' ', text)  # Remove numbers
    text = [word for word in text.split() if word not in stop_words]  # Remove stopwords
    text = ' '.join(text)
    return text

# Create k-shingles to capture context inside given texts when comparing
def k_shingles(text, k=5):
    words = text.split(' ')
    if len(words) < k:
        return set([text]) # If text is too short, return as is
    return set([' '.join(words[i:i+k]) for i in range(len(words) - k + 1)])

In [ ]:
# Combine 'Title' and 'Content' to one column - apply preprocessing and k-shingles creation
X_train = (train_data['Title'] + ' ' + train_data['Content']).apply(preprocess_text).apply(lambda x: k_shingles(x, k=1))
y_train = train_data['Label'].to_numpy()

X_test = (test_data['Title'] + ' ' + test_data['Content']).apply(preprocess_text).apply(lambda x: k_shingles(x, k=1))

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# binary=True means Binary representation: 1 if exists, 0 otherwise - for Jaccard Similarity calculation
vectorizer = CountVectorizer(binary=True, analyzer=lambda x: x)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

## Brute-Force K-Nearest Neighbors

In [6]:
import time
import numpy as np

# Number of k-Nearest Neighbors
k = 7
brute_top_k_filename = 'brute_force_top_k.txt'

start_time = time.time()

# Precompute the word counts for each training document
train_word_counts = np.array(X_train_vectorized.sum(axis=1)).ravel()
num_tests = X_test_vectorized.shape[0]

# Open temporary file to write results (in order to avoid memory overflow)
with open(brute_top_k_filename, 'w') as file:
    for i in range(num_tests):
        # Get the batch's test rows
        test_rows = X_test_vectorized[i]

        test_word_counts = test_rows.sum()
        intersection = test_rows.dot(X_train_vectorized.T).toarray().ravel()

        # Union = train_tokens + test_tokens - intersection
        union = train_word_counts + test_word_counts - intersection

        # If union = 0, turn to 1 (avoid division by 0)
        union[union == 0] = 1

        # Compute Jaccard similarity for the current batch
        jaccard_similarities = intersection / union

        # Find the indices of the top K highest similarity values
        top_k_indices = np.argpartition(-jaccard_similarities, k)[:k]

        file.write(f'{top_k_indices.tolist()}\n')

        if i % 1000 == 0:
            file.flush()

query_time = time.time() - start_time
print(f'Brute-force Query Time: {query_time:.2f} seconds')

Brute-force Query Time: 10621.21 seconds


In [7]:
# Make predictions by selecting the most common label out of the k-Nearest Neighbors (majority vote)
predictions = []
with open(brute_top_k_filename, 'r') as file:
    test_index = 0
    for line in file:
        neighbors = [int(x) for x in line.strip('[]\n').split(',')]
        neighbors = y_train[neighbors]
        labels, counts = np.unique(neighbors, return_counts=True)
        pred = labels[np.argmax(counts)]
        predictions.append(pred)

In [ ]:
# Prepare the output file for submission (test set predictions)
output_df = pd.DataFrame({'Id': test_data['Id'], 'Predicted': predictions})
output_df.to_csv('testSet_categories_brute_force_knn.csv', index=False)

## Min-Hash LSH K-Nearest Neighbors

In [9]:
from datasketch import MinHash, MinHashLSH

# Function to create MinHash from a set of shingles
def create_minhash(shingles, num_perm):
    minhash = MinHash(num_perm=num_perm)
    for shingle in shingles:
        minhash.update(shingle.encode('utf8'))
    return minhash

# Function to build the LSH index
def build_lsh_index(train_docs, num_perm, b, r, threshold=0.9):
    start_time = time.time()

    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm, params=(b, r))
    train_minhashes = {}

    # Add train documents to LSH index
    for idx, shingles in enumerate(train_docs):
        minhash = create_minhash(shingles, num_perm)
        lsh.insert(str(idx), minhash)
        train_minhashes[idx] = minhash

    build_time = time.time() - start_time
    return lsh, train_minhashes, build_time

In [ ]:
import time

# Function to perform LSH-based nearest neighbor search
def lsh_nearest_neighbors(X_test, X_test_vectorized, X_train_vectorized, lsh, num_perm, b, k=7):
    top_k_filename = f'lsh_{num_perm}_{b}_top_k.txt'
    total_comparisons = 0
    
    start_time = time.time()

    # Precompute the word counts for each training document
    train_word_counts = np.array(X_train_vectorized.sum(axis=1)).ravel()
    num_tests = X_test_vectorized.shape[0]

    # Open temporary file to write results (in order to avoid memory overflow)
    with open(top_k_filename, 'w') as file:
        for i in range(num_tests):
            # Get the batch's test row
            test_row = X_test.iloc[i]

            # Find the candidate neighbors from LSH
            minhash = create_minhash(test_row, num_perm)
            candidates = lsh.query(minhash)  # Get candidate neighbors from LSH
            candidate_indices = np.array(list(map(int, candidates)))

            # No need to compare if candidates <= k
            if len(candidate_indices) <= k:
                file.write(f'{candidate_indices.tolist()}\n')
                continue

            # Compute intersection: test_row dot product with candidate train rows
            test_row = X_test_vectorized[i]
            intersection = test_row.dot(X_train_vectorized[candidate_indices, :].T).toarray().ravel()

            total_comparisons += len(candidate_indices)

            # Compute test document's word count (sum of token counts)
            test_word_count = test_row.sum()

            # Compute train documents' word counts for candidates
            candidates_word_counts = train_word_counts[candidate_indices]

            # Compute union: train_word_counts + test_word_count - intersection
            union = candidates_word_counts + test_word_count - intersection

            # Avoid division by zero
            union[union == 0] = 1

            # Compute Jaccard similarity
            jaccard_similarities = intersection / union
            top_k = min(k, len(jaccard_similarities))

            # Find the indices of the top K highest similarity values
            top_k_indices = np.argpartition(-jaccard_similarities, top_k)[:top_k]
            top_k_indices = candidate_indices[top_k_indices]

            file.write(f'{top_k_indices.tolist()}\n')

            if i % 1000 == 0:
                file.flush()

    query_time = time.time() - start_time
    average_computed_similarities = total_comparisons/num_tests
    return top_k_filename, query_time, average_computed_similarities

In [11]:
import math

# Configurations to test (b*r=num_permutations)
configs = [
    {'num_permutations': 16, 'b': 2, 'r': 8},
    {'num_permutations': 16, 'b': 4, 'r': 4},
    {'num_permutations': 16, 'b': 8, 'r': 2},
    {'num_permutations': 32, 'b': 2, 'r': 16},
    {'num_permutations': 32, 'b': 4, 'r': 8},
    {'num_permutations': 32, 'b': 8, 'r': 4},
    {'num_permutations': 32, 'b': 16, 'r': 2},
    {'num_permutations': 64, 'b': 2, 'r': 32},
    {'num_permutations': 64, 'b': 4, 'r': 16},
    {'num_permutations': 64, 'b': 8, 'r': 8},
    {'num_permutations': 64, 'b': 16, 'r': 4},
    {'num_permutations': 64, 'b': 32, 'r': 2}
]

results = []

for config in configs:
    num_perm, b, r = config['num_permutations'], config['b'], config['r']
    print(f'Running LSH with (num_perm={num_perm}, b={b}, r={r})...')

    # Build LSH Index
    lsh, train_minhashes, lsh_build_time = build_lsh_index(
                                                X_train, 
                                                num_perm=num_perm,
                                                b=b,
                                                r=r,
                                                threshold=0.9
                                            )
    print(f'Built LSH Index in {lsh_build_time:.2f} seconds')

    # Query LSH for nearest neighbors
    lsh_neighbors_file, lsh_query_time, average_computed_similarities = lsh_nearest_neighbors(
                                                                            X_test, 
                                                                            X_test_vectorized, 
                                                                            X_train_vectorized, 
                                                                            lsh, 
                                                                            num_perm,
                                                                            b,
                                                                            k=7
                                                                        )

    print(f'Query finished after {lsh_query_time:.2f} seconds with {average_computed_similarities} similarities calculated per test document\n')

    # Calculate Matching Neighbors
    with open(brute_top_k_filename, 'r') as f1, open(lsh_neighbors_file, 'r') as f2:
        common, total = 0, 0
        for line1, line2 in zip(f1, f2):
            neighbors_brute_force = set([int(x) for x in line1.strip('[]\n').split(',')])
            neighbors_lsh = set([int(x) if x != '' else -1 for x in line2.strip('[]\n').split(',')])
            common += len(neighbors_brute_force & neighbors_lsh)
            total += len(neighbors_brute_force)

    matching_fraction = 100.0 * common / total
    expected_similarity = math.pow(1./b, 1./r)

    # Save results
    results.append([
        f'LSH-Jaccard (Perm={num_perm})',
        round(lsh_build_time, 2),
        round(lsh_query_time, 2),
        round(lsh_build_time + lsh_query_time, 2),
        f'{matching_fraction:.2f}%',
        f'Perm={num_perm}, b={b}, r={r}, τ={round(expected_similarity, 3)}'
    ])

# Add brute-force results for reference
results.insert(0, [
    'Brute-Force Jaccard', 0, round(query_time, 2), round(query_time, 2), '100%', '-'
])

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=[
    'Type', 'Build Time (sec)', 'Query Time (sec)', 'Total Time (sec)', 'Matching %', 'Parameters'
])
# Save the comparison table
results_df.to_csv('performance_comparison.csv', index=False)

Running LSH with (num_perm=16, b=2, r=8)...
Built LSH Index in 151.61 seconds
Query finished after 67.88 seconds with 0.04138837869427283 similarities calculated per test document

Running LSH with (num_perm=16, b=4, r=4)...
Built LSH Index in 166.49 seconds
Query finished after 78.04 seconds with 0.4823008849557522 similarities calculated per test document

Running LSH with (num_perm=16, b=8, r=2)...
Built LSH Index in 176.54 seconds
Query finished after 236.44 seconds with 824.2907622307564 similarities calculated per test document

Running LSH with (num_perm=32, b=2, r=16)...
Built LSH Index in 190.26 seconds
Query finished after 71.42 seconds with 0.012940390716313241 similarities calculated per test document

Running LSH with (num_perm=32, b=4, r=8)...
Built LSH Index in 194.94 seconds
Query finished after 77.53 seconds with 0.05992235765570212 similarities calculated per test document

Running LSH with (num_perm=32, b=8, r=4)...
Built LSH Index in 191.95 seconds
Query finished af

In [12]:
results_df

,Type,Build Time (sec),Query Time (sec),Total Time (sec),Matching %,Parameters
0,Brute-Force Jaccard,0.00,10621.21,10621.21,100%,-
1,LSH-Jaccard (Perm=16),151.61,67.88,219.49,3.00%,"Perm=16, b=2, r=8, τ=0.917"
2,LSH-Jaccard (Perm=16),166.49,78.04,244.53,6.33%,"Perm=16, b=4, r=4, τ=0.707"
3,LSH-Jaccard (Perm=16),176.54,236.44,412.97,32.33%,"Perm=16, b=8, r=2, τ=0.354"
4,LSH-Jaccard (Perm=32),190.26,71.42,261.67,1.86%,"Perm=32, b=2, r=16, τ=0.958"
5,LSH-Jaccard (Perm=32),194.94,77.53,272.47,3.59%,"Perm=32, b=4, r=8, τ=0.841"
6,LSH-Jaccard (Perm=32),191.95,83.06,275.01,8.10%,"Perm=32, b=8, r=4, τ=0.595"
7,LSH-Jaccard (Perm=32),185.64,287.87,473.52,46.60%,"Perm=32, b=16, r=2, τ=0.25"
8,LSH-Jaccard (Perm=64),179.61,78.57,258.17,1.11%,"Perm=64, b=2, r=32, τ=0.979"
9,LSH-Jaccard (Perm=64),180.69,76.08,256.77,2.35%,"Perm=64, b=4, r=16, τ=0.917"
